
<br>
===================================================<br>
Faces recognition example using eigenfaces and SVMs<br>
===================================================<br>
The dataset used in this example is a preprocessed excerpt of the<br>
"Labeled Faces in the Wild", aka LFW_:<br>
  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)<br>
.. _LFW: http://vis-www.cs.umass.edu/lfw/<br>
Expected results for the top 5 most represented people in the dataset:<br>
================== ============ ======= ========== =======<br>
                   precision    recall  f1-score   support<br>
================== ============ ======= ========== =======<br>
     Ariel Sharon       0.67      0.92      0.77        13<br>
     Colin Powell       0.75      0.78      0.76        60<br>
  Donald Rumsfeld       0.78      0.67      0.72        27<br>
    George W Bush       0.86      0.86      0.86       146<br>
Gerhard Schroeder       0.76      0.76      0.76        25<br>
      Hugo Chavez       0.67      0.67      0.67        15<br>
       Tony Blair       0.81      0.69      0.75        36<br>
      avg / total       0.80      0.80      0.80       322<br>
================== ============ ======= ========== =======<br>


In [ ]:
from time import time
import logging
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [ ]:
print(__doc__)

Display progress logs on stdout

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

#############################################################################<br>
Download the data, if not already on disk and load it as numpy arrays

In [ ]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

introspect the images arrays to find the shapes (for plotting)

In [ ]:
n_samples, h, w = lfw_people.images.shape

for machine learning we use the 2 data directly (as relative pixel<br>
positions info is ignored by this model)

In [ ]:
X = lfw_people.data
n_features = X.shape[1]

the label to predict is the id of the person

In [ ]:
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

In [ ]:
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

#############################################################################<br>
Split into a training set and a test set using a stratified k fold

split into a training and testing set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

#############################################################################<br>
Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled<br>
dataset): unsupervised feature extraction / dimensionality reduction

In [ ]:
n_components = 150

In [ ]:
print("Extracting the top %d eigenfaces from %d faces"
      % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

In [ ]:
eigenfaces = pca.components_.reshape((n_components, h, w))

In [ ]:
print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

#############################################################################<br>
Train a SVM classification model

In [ ]:
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(clf.best_estimator_)

#############################################################################<br>
Quantitative evaluation of the model quality on the test set

In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print("done in %0.3fs" % (time() - t0))

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))

#############################################################################<br>
Qualitative evaluation of the predictions using matplotlib

In [ ]:
def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    plt.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i + 1)
        plt.imshow(images[i].reshape((h, w)), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())

plot the result of the prediction on a portion of the test set

In [ ]:
def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

In [ ]:
prediction_titles = [title(y_pred, y_test, target_names, i)
                     for i in range(y_pred.shape[0])]

In [ ]:
plot_gallery(X_test, prediction_titles, h, w)

plot the gallery of the most significative eigenfaces

In [ ]:
eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

In [ ]:
plt.show()